In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json

In [2]:
data = json.load(open("data.json"))

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))

projects = tsdk.list_my_projects()
dd.display(projects)

== SDK package is up to date ==


                                        Projects                                         
┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Index ┃ Name  ┃ Id                                   ┃ Created at       ┃ Description ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 0     │ ertsi │ bc91d45c-e9f6-410f-ba72-d0fbfffcb844 │ 2025-05-11 17:38 │             │
└───────┴───────┴──────────────────────────────────────┴──────────────────┴─────────────┘

In [3]:
project = tsdk.get_or_create_project("ertsi")

In [4]:
edge_points_hexagon = data["room_verts"]
room = treble.GeometryDefinitionGenerator.create_polygon_room(
    points_xy=edge_points_hexagon, height_z=3, join_wall_layers=False
)

In [5]:
room.clear_geometry_components()
placement = treble.GeometryComponentPlacement(
    components=tsdk.geometry_component_library.query(group="desk"),
    preferred_count=3,
    rotation_settings=treble.ComponentAnglePool([0]),
)
room.populate_with_geometry_components(
    components=[placement],
    selection_algorithm=treble.ComponentSelectionAlgorithm.random,
)

In [6]:
try:
    model = project.add_model("room1", room)
except Exception as e:
    print(f"Model already exists: {e}")
    model = project.get_model_by_name("room1")
assert model, "Model not found in project"

One or more validation errors occurred.
Field: "name" - ["Model named 'room1' already exists, please select another name."]


Model already exists: 'NoneType' object has no attribute 'id'


In [9]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]
layers = {
    "polygon_room_wall_0": data["wall_0_material"],
    "polygon_room_wall_1": data["wall_1_material"],
    "polygon_room_wall_2": data["wall_2_material"],
    "polygon_room_wall_3": data["wall_3_material"],
    "polygon_room_floor": data["floor_material"],
    "polygon_room_ceiling": data["ceiling_material"],
    "Furniture/Desk": "wood",
}
material_assignment = []

assert model.layer_names, "No layers found in model"
for layer in model.layer_names:
    if layer in layers:
        search_string = layers[layer]
        matches = [
            m for m in database_materials if search_string.lower() in m.name.lower()
        ]
        if matches:
            material_assignment.append(
                treble.MaterialAssignment(layer, random.choice(matches))
            )

# show the material assignment
dd.display(material_assignment)

                                   Material assignment                                   
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ polygon_room_wall_1  │ Smooth painted concrete                                        │
│ polygon_room_wall_0  │ Window glass                                                   │
│ polygon_room_wall_2  │ Smooth painted concrete                                        │
│ polygon_room_wall_3  │ Smooth painted concrete                                        │
│ polygon_room_floor   │ Linoleum or vinyl on concrete                                  │
│ polygon_room_ceiling │ Slotted panel, 17% open, 40 mm 81 kg/m3 absorber, 30 mm cavity │
│ Furniture/Desk       │ Wood panel with air space behind                               │
└──────────────────────┴────────────────────────────────────────────────────────────────┘

In [10]:
room.plot()

In [15]:
source = treble.Source.make_omni(
    position=treble.Point3d(0.25, 0.5, 1.5), label="source_1"
)
receiver = treble.Receiver.make_mono(
    position=treble.Point3d(3, 5, 1.2), label="receiver_1"
)
sim_def = treble.SimulationDefinition(
    name="Simulation_1",  # unique name of the simulation
    simulation_type=treble.SimulationType.ga,  # the type of simulation
    model=model,  # the model we created in an earlier step
    energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -40 dB of energy decay
    receiver_list=[receiver],
    source_list=[source],
    material_assignment=material_assignment,
)

sim_def.plot()

In [16]:
simulation = project.add_simulation(sim_def)

In [17]:
simulation.start()
simulation.as_live_progress()

Simulation_1
├── Id: 36276580-5214-43ca-bc96-60cce4c07c45
├── Status: Completed
├── Percentage completed: 100%
└── Sources
    └── source_1 (3330a9ad-20f9-4005-a84a-5c73f0023a7c)
        ├── Type: SourceType.omni
        └── Tasks
            ├── Id: 436b342e-fdd3-4c81-8c66-03d4f34e7093
            ├── Type: TaskType.ga
            ├── Status: Completed
            └── Percentage completed: 100%

Done

In [18]:
results_object = simulation.download_results(f'results/{simulation.name}')

# begin to explore the results
results_object.plot()

/home/leo/projects/repos/ertsi/generation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Download simulation 36276580-5214-43ca-bc96-60cce4c07c45: 100%|██████████| 2/2 tasks


In [19]:
# isolate the data from a single impulse response
this_result = results_object.get_mono_ir(source=simulation.sources[0],receiver=simulation.receivers[0])
this_ir = this_result.data
this_tvec = this_result.time

this_result.plot()